# Getting Started!
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/understandable-machine-intelligence-lab/Quantus/main?labpath=tutorials%2FTutorial_Getting_Started_with_Time_Series_Data.ipynb)


This notebook shows how to get started with Quantus using time series data.

For this purpose, we use the 1D analogue of the [MNIST dataset](https://github.com/greydanus/mnist1d) (authored, Sam Greydanus) and a [CNN model](https://github.com/greydanus/mnist1d/blob/master/models.py#L36) taken from the same repository.

In [1]:
from IPython.display import clear_output

!python -m pip install quantus torch captum
clear_output()

In [3]:
import pickle
import urllib.request
import numpy as np

import quantus
from captum.attr import IntegratedGradients
import torch
import torch.nn as nn

torch.manual_seed(27)

# Set seeds.
clear_output()
np.random.seed(27)

## 1) Preliminaries

### 1.1 Load datasets

We load the dataset via urllib and pickle.
Alternatively, it can be downloaded directly from the github repository: https://github.com/greydanus/mnist1d

In [4]:
mnist_url = "https://github.com/greydanus/mnist1d/blob/master/mnist1d_data.pkl?raw=true"
mnist1d_data = pickle.load(urllib.request.urlopen(mnist_url))
mnist1d_data.keys()

dict_keys(['x', 'x_test', 'y', 'y_test', 't', 'templates'])

In [5]:
train_features = mnist1d_data["x"]
train_labels = mnist1d_data["y"]
test_features = mnist1d_data["x_test"]
test_labels = mnist1d_data["y_test"]

print(f"{train_features.shape = }")
print(f"{train_labels.shape = }")
print(f"{test_features.shape = }")
print(f"{test_labels.shape = }")

train_features.shape = (4000, 40)
train_labels.shape = (4000,)
test_features.shape = (1000, 40)
test_labels.shape = (1000,)


### 1.2 Train a model

The model is based on the model provided by the mnist1d repository:

https://github.com/greydanus/mnist1d/blob/master/models.py#L36

In [6]:
class ConvBase(nn.Module):
    def __init__(self, output_size, channels=25, linear_in=125):
        super(ConvBase, self).__init__()
        self.conv1 = nn.Conv1d(1, channels, 5, stride=2, padding=1)
        self.conv2 = nn.Conv1d(channels, channels, 3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(channels, channels, 3, stride=2, padding=1)
        self.linear = nn.Linear(
            linear_in, output_size
        )  # flattened channels -> 10 (assumes input has dim 50)
        print(
            "Initialized ConvBase model with {} parameters".format(self.count_params())
        )

    def count_params(self):
        return sum([p.view(-1).shape[0] for p in self.parameters()])

    def forward(self, x, verbose=False):
        x = x.view(-1, 1, x.shape[-1])
        h1 = self.conv1(x).relu()
        h2 = self.conv2(h1).relu()
        h3 = self.conv3(h2).relu()
        h3 = h3.view(h3.shape[0], -1)
        return self.linear(h3)

In [7]:
# Load the model.
net = ConvBase(output_size=40)

# Set training configs.
criterion = nn.CrossEntropyLoss()
num_epochs = 200
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
input_tensor = torch.from_numpy(train_features).type(torch.FloatTensor)
label_tensor = torch.from_numpy(train_labels)

# Train model!
for epoch in range(num_epochs):
    output = net(input_tensor)
    loss = criterion(output, label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print("Epoch {}/{} => Loss: {:.2f}".format(epoch + 1, num_epochs, loss.item()))

Initialized ConvBase model with 8990 parameters
Epoch 1/200 => Loss: 3.71
Epoch 21/200 => Loss: 1.50
Epoch 41/200 => Loss: 1.12
Epoch 61/200 => Loss: 1.39
Epoch 81/200 => Loss: 0.66
Epoch 101/200 => Loss: 0.46
Epoch 121/200 => Loss: 0.38
Epoch 141/200 => Loss: 0.35
Epoch 161/200 => Loss: 0.20
Epoch 181/200 => Loss: 0.75


In [8]:
# Reformat the train set predictions.
out_probs = net(input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Train Accuracy:", sum(out_classes == train_labels) / len(train_labels))

Train Accuracy: 0.865


In [9]:
# Reformat the test set predictions.
test_input_tensor = torch.from_numpy(test_features).type(torch.FloatTensor)
out_probs = net(test_input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Test Accuracy:", sum(out_classes == test_labels) / len(test_labels))

Test Accuracy: 0.812


### 1.3 Generate explanations

In this example, we rely on the `captum` library. We use the Integrated Gradients method.

In [10]:
# Load Integrated Gradients.
ig = IntegratedGradients(net)

# Reformat attributions.
test_input_tensor.requires_grad_()
attr, delta = ig.attribute(test_input_tensor, target=1, return_convergence_delta=True)
attr = attr.detach().numpy()

## 2) Quantative evaluation using Quantus

We can evaluate our explanations on a variety of quantuative criteria but as a motivating example we test the ModelParameterRandomisation scores by Adebayo et al., 2018 and Complexity Bhatt et al., 2020.

The ModelParameterRandomisation metric measures the distance between the original attribution and a newly computed attribution throughout the process of cascadingly/independently randomizing the model parameters of one layer at a time.

The Complexity of attributions is defined as the entropy of the fractional contribution of feature x_i to the total
magnitude of the attribution.

In [11]:
# For Quantus, make sure to reshape the data by adding a (1) channel
test_features = test_features.reshape(1000, 1, 40)

In [12]:
# Return ModelParameterRandomisation scores for Integrated Gradients.
scores_intgrad = quantus.ModelParameterRandomisation(
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_sample_correlation=True,
    return_aggregate=True,
    aggregate_func=np.mean,
    layer_order="independent",
    disable_warnings=True,
    normalise=True,
    abs=True,
    display_progressbar=True,
)(
    model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={
        "method": "IntegratedGradients",
    },
)
print(
    f"ModelParameterRandomisation scores by Adebayo et al., 2018\n"
    f"\n • Integrated Gradient = ",
    scores_intgrad,
)

  0%|          | 0/4 [00:00<?, ?it/s]

ModelParameterRandomisation scores by Adebayo et al., 2018

 • Integrated Gradient =  [0.7655924997956273]


In [13]:
complexity_intgrad = quantus.Complexity(
    normalise=True,
    abs=True,
    disable_warnings=True,
    display_progressbar=True,
    return_aggregate=True,
)(
    model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "IntegratedGradients", "reduce_axes": ()},
)

print(
    f"Complexity Bhatt et al., 2020.\n" f"\n • Integrated Gradient = ",
    complexity_intgrad,
)

Evaluating Complexity:   0%|          | 0/1000 [00:00<?, ?it/s]

Complexity Bhatt et al., 2020.

 • Integrated Gradient =  [1.922484145167385]
